# Pairwise distance analysis with Plink

You have already seen one way to visualise genotype data in low dimensions, using smartPCA. We now turn to another method, called MDS and implemented in the software package [Plink](https://zzz.bwh.harvard.edu/plink/).

We will first have to convert the EIGENSTRAT-formatted genotype data that we produced via `trident forge` to Plink format. We can do that using `trident genoconvert`:

In [1]:
!trident genoconvert -d forged_package --outFormat PLINK

Searching POSEIDON.yml files... 1 found
Initializing packages... 
> 1 
Packages loaded: 1
Converting genotype data in package PCA_package_1 to format PLINK:
Processing SNPs...
> 0 > 1000 > 2000 > 3000 > 4000 > 5000 > 6000 > 7000 > 8000 > 9000 > 10000 > 11000 > 12000 > 13000 > 14000 > 15000 > 16000 > 17000 > 18000 > 19000 > 20000 > 21000 > 22000 > 23000 > 24000 > 25000 > 26000 > 27000 > 28000 > 29000 > 30000 > 31000 > 32000 > 33000 > 34000 > 35000 > 36000 > 37000 > 38000 > 39000 > 40000 > 41000 > 42000 > 43000 > 44000 > 45000 > 46000 > 47000 > 48000 > 49000 > 50000 > 51000 > 52000 > 53000 > 54000 > 55000 > 56000 > 57000 > 58000 > 59000 > 60000 > 61000 > 62000 > 63000 > 64000 > 65000 > 66000 > 67000 > 68000 > 69000 > 70000 > 71000 > 72000 > 73000 > 74000 > 75000 > 76000 > 77000 > 78000 > 79000 > 80000 > 81000 > 82000 > 83000 > 84000 > 85000 > 86000 > 87000 > 88000 > 89000 > 90000 > 91000 > 92000 > 93000 > 94000 > 95000 > 96000 > 97000 > 98000 > 99000 > 100000 > 101000 > 102000 > 103000 >

We can now find the PLINK data in the same directory as the EIGENSTRAT data:

In [2]:
!ls forged_package/

PCA_package_1.bed   PCA_package_1.fam   PCA_package_1.janno
PCA_package_1.bib   PCA_package_1.geno  PCA_package_1.snp
PCA_package_1.bim   PCA_package_1.ind   POSEIDON.yml


Where the `.bed`, `.bim` and `.fam` files are used in PLINK. In fact, they are automatically used whenever you use the option `--bfile forged_package/PCA_package` in plink. The first command we're gonna use is for computing pairwise distances of all individuals:

In [4]:
!plink --bfile forged_package/PCA_package_1 --genome --out pairwise_distances

PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to pairwise_distances.log.
Options in effect:
  --bfile forged_package/PCA_package_1
  --genome
  --out pairwise_distances

16384 MB RAM detected; reserving 8192 MB for main workspace.
597573 variants loaded from .bim file.
1666 people (1151 males, 504 females, 11 ambiguous) loaded from .fam.
Ambiguous sex IDs written to pairwise_distances.nosex .
Using up to 4 threads (change this with --threads).
Before main variant filters, 1666 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
treat these as missing.
Total genotyping rate is 0.994538.
597573 variants and 1666 people pass filters and QC.
Note: No phenotypes present.
E